# Boston Property Data:


url = 'https://data.boston.gov/api/3/action/datastore_search?resource_id=4b99718b-d064-471b-9b24-517ae5effecc'

### Set up environment

In [1]:
import pandas as pd 
import numpy as np 
from numpy import NaN
import urllib.request
import requests
import io
import csv

In [2]:
# set up notebook to display multiple output in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
 
# Set option to format printing of Float columns
pd.options.display.float_format = '{:.2f}'.format

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
# Mount Google Drive

from google.colab import drive
#drive.mount('/content/gdrive')
drive.mount('/gdrive') 
%cd /gdrive/My\ Drive/MSDS\ 498\ Capstone/4.\ Analytics/4a.\ Data

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/MSDS 498 Capstone/4. Analytics/4a. Data


# Pull 2022 Boston Property Assessments

In [4]:

# url = 'https://data.boston.gov/api/3/action/datastore_search?resource_id=4b99718b-d064-471b-9b24-517ae5effecc'
# import data sets for 2022 and 2021 and 2020 and 2019
url2022= 'https://data.boston.gov/dataset/e02c44d2-3c64-459c-8fe2-e1ce5f38a035/resource/4b99718b-d064-471b-9b24-517ae5effecc/download/fy2022pa-4.csv'  


pa2022alldf = pd.read_csv(url2022, low_memory=False)
pa2022alldf["filename"] = '2022'
pa2022alldf["PID"] = pa2022alldf["PID"].astype(str)
 
print(pa2022alldf.shape)


(178598, 65)


In [5]:
# Limit to Owner Occupied, Single-Family  
pa2022df = pa2022alldf[( pa2022alldf["LU"]=='R1')  & (pa2022alldf["OWN_OCC"]=="Y")]

print(pa2022df.shape)
print(pa2022df.groupby(by="OWN_OCC")["OWN_OCC"].count() )
print(pa2022df.groupby(by="LU")["LU"].count() )
pa2022df.info()

(25640, 65)
OWN_OCC
Y    25640
Name: OWN_OCC, dtype: int64
LU
R1    25640
Name: LU, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25640 entries, 27 to 178588
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              25640 non-null  object 
 1   CM_ID            25640 non-null  object 
 2   GIS_ID           25640 non-null  float64
 3   ST_NUM           25640 non-null  object 
 4   ST_NAME          25640 non-null  object 
 5   UNIT_NUM         1 non-null      object 
 6   CITY             25640 non-null  object 
 7   ZIPCODE          25640 non-null  int64  
 8   BLDG_SEQ         25640 non-null  float64
 9   NUM_BLDGS        25640 non-null  int64  
 10  LUC              25640 non-null  int64  
 11  LU               25640 non-null  object 
 12  LU_DESC          25640 non-null  object 
 13  BLDG_TYPE        25640 non-null  object 
 14  OWN_OCC          25640 non-null  object 
 15  OWNER

### Limit To Features of Interest

In [6]:
# Create List of features to keep
keeplist = [ 'PID'
, 'YR_REMODEL'
, 'RES_FLOOR'
, 'GROSS_AREA'
, 'LIVING_AREA'
, 'INT_COND'
, 'EXT_COND'
, 'OVERALL_COND'
, 'BED_RMS'
, 'FULL_BTH'
, 'HLF_BTH'
, 'KITCHEN'
, 'TT_RMS'
, 'BTHRM_STYLE1'
, 'BTHRM_STYLE2'
, 'KITCHEN_TYPE'
, 'KITCHEN_STYLE1' 
]
# Select the Required  Columns
pa2022df = pa2022df[keeplist]

pa2022df.info()
pa2022df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25640 entries, 27 to 178588
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PID             25640 non-null  object 
 1   YR_REMODEL      10467 non-null  float64
 2   RES_FLOOR       25640 non-null  float64
 3   GROSS_AREA      25640 non-null  float64
 4   LIVING_AREA     25640 non-null  float64
 5   INT_COND        25640 non-null  object 
 6   EXT_COND        25640 non-null  object 
 7   OVERALL_COND    25640 non-null  object 
 8   BED_RMS         25640 non-null  int64  
 9   FULL_BTH        25640 non-null  int64  
 10  HLF_BTH         25640 non-null  int64  
 11  KITCHEN         25640 non-null  int64  
 12  TT_RMS          25640 non-null  int64  
 13  BTHRM_STYLE1    25640 non-null  object 
 14  BTHRM_STYLE2    19580 non-null  object 
 15  KITCHEN_TYPE    25639 non-null  object 
 16  KITCHEN_STYLE1  25640 non-null  object 
dtypes: float64(4), int64(5), obje

,YR_REMODEL,RES_FLOOR,GROSS_AREA,LIVING_AREA,BED_RMS,FULL_BTH,HLF_BTH,KITCHEN,TT_RMS
count,10467.00,25640.00,25640.00,25640.00,25640.00,25640.00,25640.00,25640.00,25640.00
mean,2004.96,1.87,2782.37,1755.67,3.39,1.48,0.59,1.03,7.26
std,13.63,0.57,925.84,696.52,0.96,0.69,0.55,0.17,1.74
min,1900.00,1.00,570.00,372.00,0.00,1.00,0.00,1.00,2.00
25%,2000.00,1.50,2192.00,1320.00,3.00,1.00,0.00,1.00,6.00
50%,2008.00,2.00,2578.00,1602.00,3.00,1.00,1.00,1.00,7.00
75%,2015.00,2.00,3151.00,2015.05,4.00,2.00,1.00,1.00,8.00
max,2021.00,19.00,22802.00,20326.00,10.00,12.00,4.00,3.00,27.00


In [7]:
# Check the values of All Columns

# Get a list of all columns
collist = list(pa2022df.columns)
print(collist)
print(len(collist))

# Remove PID and YR_REMODEL from the list
collist = collist[2:]
print(collist)
print(len(collist))

# Print Frequencies of all values
for col in collist:
  print(pa2022df[col].value_counts().sort_index() )

['PID', 'YR_REMODEL', 'RES_FLOOR', 'GROSS_AREA', 'LIVING_AREA', 'INT_COND', 'EXT_COND', 'OVERALL_COND', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'KITCHEN', 'TT_RMS', 'BTHRM_STYLE1', 'BTHRM_STYLE2', 'KITCHEN_TYPE', 'KITCHEN_STYLE1']
17
['RES_FLOOR', 'GROSS_AREA', 'LIVING_AREA', 'INT_COND', 'EXT_COND', 'OVERALL_COND', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'KITCHEN', 'TT_RMS', 'BTHRM_STYLE1', 'BTHRM_STYLE2', 'KITCHEN_TYPE', 'KITCHEN_STYLE1']
15
1.00      4677
1.50      2894
2.00     14922
2.50      1537
3.00      1261
3.50        84
4.00       227
4.50        16
5.00        21
19.00        1
Name: RES_FLOOR, dtype: int64
570.00      1
632.00      1
676.00      1
691.20      1
763.20      1
           ..
13128.60    1
13573.10    1
13720.10    1
16901.30    1
22802.00    1
Name: GROSS_AREA, Length: 11825, dtype: int64
372.00      1
403.20      1
470.00      1
480.60      1
489.60      1
           ..
10080.45    1
10198.45    1
10995.00    1
13033.00    1
20326.00    1
Name: LIVING_AREA, Length: 7542

### Convert Categorical Values to Ranks

In [8]:
pa2022moddf = pa2022df.copy() 

pa2022moddf['INT_COND_RANK'] = NaN
pa2022moddf['INT_COND_RANK'] = np.where(pa2022moddf['INT_COND'] == 'P - Poor'      , 1 , pa2022moddf['INT_COND_RANK'] )      
pa2022moddf['INT_COND_RANK'] = np.where(pa2022moddf['INT_COND'] == 'F - Fair'      , 2 , pa2022moddf['INT_COND_RANK'] )   
pa2022moddf['INT_COND_RANK'] = np.where(pa2022moddf['INT_COND'] == 'A - Average'   , 3 , pa2022moddf['INT_COND_RANK'] )
pa2022moddf['INT_COND_RANK'] = np.where(pa2022moddf['INT_COND'] == 'G - Good'      , 4 , pa2022moddf['INT_COND_RANK'] )
pa2022moddf['INT_COND_RANK'] = np.where(pa2022moddf['INT_COND'] == 'E - Excellent' , 5 , pa2022moddf['INT_COND_RANK'] ) 

pa2022moddf['EXT_COND_RANK'] = NaN
pa2022moddf['EXT_COND_RANK'] = np.where(pa2022moddf['EXT_COND'] == 'P - Poor'      , 1 , pa2022moddf['EXT_COND_RANK'] )      
pa2022moddf['EXT_COND_RANK'] = np.where(pa2022moddf['EXT_COND'] == 'F - Fair'      , 2 , pa2022moddf['EXT_COND_RANK'] )   
pa2022moddf['EXT_COND_RANK'] = np.where(pa2022moddf['EXT_COND'] == 'A - Average'   , 3 , pa2022moddf['EXT_COND_RANK'] )
pa2022moddf['EXT_COND_RANK'] = np.where(pa2022moddf['EXT_COND'] == 'G - Good'      , 4 , pa2022moddf['EXT_COND_RANK'] )
pa2022moddf['EXT_COND_RANK'] = np.where(pa2022moddf['EXT_COND'] == 'E - Excellent' , 5 , pa2022moddf['EXT_COND_RANK'] ) 

pa2022moddf['OVERALL_COND_RANK'] = NaN
pa2022moddf['OVERALL_COND_RANK'] = np.where(pa2022moddf['OVERALL_COND'] == 'P - Poor'      , 1 , pa2022moddf['OVERALL_COND_RANK'] )      
pa2022moddf['OVERALL_COND_RANK'] = np.where(pa2022moddf['OVERALL_COND'] == 'F - Fair'      , 2 , pa2022moddf['OVERALL_COND_RANK'] )   
pa2022moddf['OVERALL_COND_RANK'] = np.where(pa2022moddf['OVERALL_COND'] == 'A - Average'   , 3 , pa2022moddf['OVERALL_COND_RANK'] )
pa2022moddf['OVERALL_COND_RANK'] = np.where(pa2022moddf['OVERALL_COND'] == 'G - Good'      , 4 , pa2022moddf['OVERALL_COND_RANK'] )
pa2022moddf['OVERALL_COND_RANK'] = np.where(pa2022moddf['OVERALL_COND'] == 'E - Excellent' , 5 , pa2022moddf['OVERALL_COND_RANK'] ) 

pa2022moddf['BTHRM_STYLE1_RANK'] = NaN
pa2022moddf['BTHRM_STYLE1_RANK'] = np.where(pa2022moddf['BTHRM_STYLE1'] == 'N - No Remodeling' , 1 , pa2022moddf['BTHRM_STYLE1_RANK'] )      
pa2022moddf['BTHRM_STYLE1_RANK'] = np.where(pa2022moddf['BTHRM_STYLE1'] == 'S - Semi-Modern'   , 2 , pa2022moddf['BTHRM_STYLE1_RANK'] )   
pa2022moddf['BTHRM_STYLE1_RANK'] = np.where(pa2022moddf['BTHRM_STYLE1'] == 'M - Modern'        , 3 , pa2022moddf['BTHRM_STYLE1_RANK'] )
pa2022moddf['BTHRM_STYLE1_RANK'] = np.where(pa2022moddf['BTHRM_STYLE1'] == 'L - Luxury'        , 4 , pa2022moddf['BTHRM_STYLE1_RANK'] ) 

pa2022moddf['BTHRM_STYLE2_RANK'] = NaN
pa2022moddf['BTHRM_STYLE2_RANK'] = np.where(pa2022moddf['BTHRM_STYLE2'] == 'N - No Remodeling' , 1 , pa2022moddf['BTHRM_STYLE2_RANK'] )      
pa2022moddf['BTHRM_STYLE2_RANK'] = np.where(pa2022moddf['BTHRM_STYLE2'] == 'S - Semi-Modern'   , 2 , pa2022moddf['BTHRM_STYLE2_RANK'] )   
pa2022moddf['BTHRM_STYLE2_RANK'] = np.where(pa2022moddf['BTHRM_STYLE2'] == 'M - Modern'        , 3 , pa2022moddf['BTHRM_STYLE2_RANK'] )
pa2022moddf['BTHRM_STYLE2_RANK'] = np.where(pa2022moddf['BTHRM_STYLE2'] == 'L - Luxury'        , 4 , pa2022moddf['BTHRM_STYLE2_RANK'] ) 

pa2022moddf['KITCHEN_STYLE1_RANK'] = NaN
pa2022moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2022moddf['KITCHEN_STYLE1'] == 'N - No Remodeling' , 1 , pa2022moddf['KITCHEN_STYLE1_RANK'] )      
pa2022moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2022moddf['KITCHEN_STYLE1'] == 'S - Semi-Modern'   , 2 , pa2022moddf['KITCHEN_STYLE1_RANK'] )   
pa2022moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2022moddf['KITCHEN_STYLE1'] == 'M - Modern'        , 3 , pa2022moddf['KITCHEN_STYLE1_RANK'] )
pa2022moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2022moddf['KITCHEN_STYLE1'] == 'L - Luxury'        , 4 , pa2022moddf['KITCHEN_STYLE1_RANK'] ) 

pa2022moddf['KITCHEN_TYPE_RANK'] = NaN
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == 'N - None'                    , 0 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == 'O - One Person'              , 0 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == '0F - 0 Full Eat In Kitchens' , 0 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == 'F - Full Eat In'             , 1 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == '1F - 1 Full Eat In Kitchens' , 1 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == '2F - 2 Full Eat In Kitchens' , 2 , pa2022moddf['KITCHEN_TYPE_RANK']  )  
pa2022moddf['KITCHEN_TYPE_RANK'] = np.where(pa2022moddf['KITCHEN_TYPE'] == '3F - 3 Full Eat In Kitchens' , 3 , pa2022moddf['KITCHEN_TYPE_RANK']  )  

def rankchk(orig, rnk):
  print('\n============================================')
  print(orig)
  print('============================================\n')
  print(pa2022moddf[[rnk, orig]].value_counts().sort_index() )

rankchk('INT_COND'       ,'INT_COND_RANK')
rankchk('EXT_COND'       , 'EXT_COND_RANK')
rankchk('OVERALL_COND'   , 'OVERALL_COND_RANK')
rankchk('BTHRM_STYLE1'   , 'BTHRM_STYLE1_RANK')
rankchk('BTHRM_STYLE2'   , 'BTHRM_STYLE2_RANK')
rankchk('KITCHEN_STYLE1' , 'KITCHEN_STYLE1_RANK')
rankchk('KITCHEN_TYPE'   , 'KITCHEN_TYPE_RANK')



INT_COND

INT_COND_RANK  INT_COND     
1.00           P - Poor            18
2.00           F - Fair           312
3.00           A - Average      16300
4.00           G - Good          8603
5.00           E - Excellent      407
dtype: int64

EXT_COND

EXT_COND_RANK  EXT_COND     
1.00           P - Poor            19
2.00           F - Fair           579
3.00           A - Average      19040
4.00           G - Good          5967
5.00           E - Excellent       35
dtype: int64

OVERALL_COND

OVERALL_COND_RANK  OVERALL_COND 
1.00               P - Poor            12
2.00               F - Fair           206
3.00               A - Average      19214
4.00               G - Good          6010
5.00               E - Excellent      198
dtype: int64

BTHRM_STYLE1

BTHRM_STYLE1_RANK  BTHRM_STYLE1     
1.00               N - No Remodeling     1574
2.00               S - Semi-Modern      14484
3.00               M - Modern            9027
4.00               L - Luxury             555
dtype: 

### Drop the Original Columns (Keep the Ranks) And Rename All Columns

In [9]:
keepcols = [
  'PID'
, 'YR_REMODEL'
, 'RES_FLOOR'
, 'GROSS_AREA'
, 'LIVING_AREA'
, 'BED_RMS'
, 'FULL_BTH'
, 'HLF_BTH'
, 'KITCHEN'
, 'TT_RMS'
, 'INT_COND_RANK'
, 'EXT_COND_RANK'
, 'OVERALL_COND_RANK'
, 'BTHRM_STYLE1_RANK'
, 'BTHRM_STYLE2_RANK'
, 'KITCHEN_STYLE1_RANK'
, 'KITCHEN_TYPE_RANK'
]

# Add '22' Suffix to All Column Names
keepcols22 = [sub + '_22' for sub in keepcols]

print(keepcols)
print(keepcols22)

# Create Final 2022 DF with Select Columns
pa2022finldf = pa2022moddf[keepcols]
pa2022finldf.columns = keepcols22
pa2022finldf.info()


['PID', 'YR_REMODEL', 'RES_FLOOR', 'GROSS_AREA', 'LIVING_AREA', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'KITCHEN', 'TT_RMS', 'INT_COND_RANK', 'EXT_COND_RANK', 'OVERALL_COND_RANK', 'BTHRM_STYLE1_RANK', 'BTHRM_STYLE2_RANK', 'KITCHEN_STYLE1_RANK', 'KITCHEN_TYPE_RANK']
['PID_22', 'YR_REMODEL_22', 'RES_FLOOR_22', 'GROSS_AREA_22', 'LIVING_AREA_22', 'BED_RMS_22', 'FULL_BTH_22', 'HLF_BTH_22', 'KITCHEN_22', 'TT_RMS_22', 'INT_COND_RANK_22', 'EXT_COND_RANK_22', 'OVERALL_COND_RANK_22', 'BTHRM_STYLE1_RANK_22', 'BTHRM_STYLE2_RANK_22', 'KITCHEN_STYLE1_RANK_22', 'KITCHEN_TYPE_RANK_22']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25640 entries, 27 to 178588
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PID_22                  25640 non-null  object 
 1   YR_REMODEL_22           10467 non-null  float64
 2   RES_FLOOR_22            25640 non-null  float64
 3   GROSS_AREA_22           25640 non-null  float64


# Pull 2017 Boston Property Assessments

In [10]:
url2017= 'https://data.boston.gov/dataset/e02c44d2-3c64-459c-8fe2-e1ce5f38a035/resource/062fc6fa-b5ff-4270-86cf-202225e40858/download/property-assessment-fy2017.csv'

pa2017df = pd.read_csv(url2017, low_memory=False)
pa2017df["filename"] = '2017'
pa2017df["PID"] = pa2017df["PID"].astype(str)
 
print(pa2017df.shape)
print(pa2017df.info())



(170910, 76)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170910 entries, 0 to 170909
Data columns (total 76 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   PID              170910 non-null  object 
 1   CM_ID            76407 non-null   object 
 2   GIS_ID           170910 non-null  object 
 3   ST_NUM           170910 non-null  object 
 4   ST_NAME          170910 non-null  object 
 5   ST_NAME_SUF      170521 non-null  object 
 6   UNIT_NUM         62976 non-null   object 
 7   ZIPCODE          170910 non-null  object 
 8   PTYPE            170910 non-null  int64  
 9   LU               170910 non-null  object 
 10  OWN_OCC          170910 non-null  object 
 11  OWNER            170910 non-null  object 
 12  MAIL_ADDRESSEE   170910 non-null  object 
 13  MAIL_ADDRESS     170910 non-null  object 
 14  MAIL CS          170910 non-null  object 
 15  MAIL_ZIPCODE     170910 non-null  object 
 16  AV_LAND          170910 n

## Limit to Features of Interest and Rename to Match 2022

In [11]:
# (NOTE: KITCHEN_TYPE does NOT exist)

keepcols = [
 'PID'
,'YR_REMOD'
,'GROSS_AREA'
,'LIVING_AREA'
,'NUM_FLOORS'
,'R_TOTAL_RMS'
,'R_BDRMS'
,'R_FULL_BTH'
,'R_HALF_BTH'
,'R_BTH_STYLE'
,'R_BTH_STYLE2'
,'R_KITCH'
,'R_KITCH_STYLE'
,'R_EXT_CND'
,'R_OVRALL_CND'
,'R_INT_CND'
]

# rename columns to match 2022 (NOTE: KITCHEN_TYPE does NOT exist)
renamecols = [ 
 'PID'
,'YR_REMODEL'
,'GROSS_AREA'
,'LIVING_AREA'
,'RES_FLOOR'
,'TT_RMS'
,'BED_RMS'
,'FULL_BTH'
,'HLF_BTH'
,'BTHRM_STYLE1'
,'BTHRM_STYLE2'
,'KITCHEN'
,'KITCHEN_STYLE1'
,'EXT_COND'
,'OVERALL_COND'
,'INT_COND'   
]


# Select the Required  Columns
pa2017df = pa2017df[keepcols]
print(pa2017df.info() )

# Rename columns
pa2017df.columns = renamecols
print(pa2017df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170910 entries, 0 to 170909
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   PID            170910 non-null  object 
 1   YR_REMOD       131370 non-null  float64
 2   GROSS_AREA     166652 non-null  float64
 3   LIVING_AREA    166652 non-null  float64
 4   NUM_FLOORS     149101 non-null  float64
 5   R_TOTAL_RMS    62863 non-null   float64
 6   R_BDRMS        62863 non-null   float64
 7   R_FULL_BTH     62863 non-null   float64
 8   R_HALF_BTH     62863 non-null   float64
 9   R_BTH_STYLE    62863 non-null   object 
 10  R_BTH_STYLE2   54441 non-null   object 
 11  R_KITCH        62863 non-null   float64
 12  R_KITCH_STYLE  62863 non-null   object 
 13  R_EXT_CND      62863 non-null   object 
 14  R_OVRALL_CND   62863 non-null   object 
 15  R_INT_CND      62863 non-null   object 
dtypes: float64(9), object(7)
memory usage: 20.9+ MB
None
<class 'pandas.core.f

In [12]:
print(pa2017df.shape)

# Drop Duplicates
pa2017df = pa2017df.drop_duplicates(subset="PID")

print(pa2017df.shape)


(170910, 16)
(168181, 16)


In [13]:
# Check the values of All Columns

# Get a list of all columns
collist = list(pa2017df.columns)
print(collist)
print(len(collist))

# Remove PID from the list
collist = collist[1:]
print(collist)
print(len(collist))

# Print Frequencies of all values
for col in collist:
  print(pa2017df[col].value_counts().sort_index() )

['PID', 'YR_REMODEL', 'GROSS_AREA', 'LIVING_AREA', 'RES_FLOOR', 'TT_RMS', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'BTHRM_STYLE1', 'BTHRM_STYLE2', 'KITCHEN', 'KITCHEN_STYLE1', 'EXT_COND', 'OVERALL_COND', 'INT_COND']
16
['YR_REMODEL', 'GROSS_AREA', 'LIVING_AREA', 'RES_FLOOR', 'TT_RMS', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'BTHRM_STYLE1', 'BTHRM_STYLE2', 'KITCHEN', 'KITCHEN_STYLE1', 'EXT_COND', 'OVERALL_COND', 'INT_COND']
15
0.00       47229
995.00         1
1500.00        1
1828.00        1
1878.00        1
           ...  
2012.00     2269
2013.00     2181
2014.00     2181
2015.00     3025
2016.00      307
Name: YR_REMODEL, Length: 104, dtype: int64
0.00          26248
1.00            572
2.00              1
3.00              2
25.00             1
              ...  
1493304.00        1
1502842.00        1
1645931.00        1
1970176.00        1
1976650.00        1
Name: GROSS_AREA, Length: 12557, dtype: int64
0.00          26252
1.00            575
2.00              3
3.00              1
25.00 

### Convert Categorical Values to Ranks

In [14]:
pa2017moddf = pa2017df.copy() 

pa2017moddf['INT_COND_RANK'] = NaN
pa2017moddf['INT_COND_RANK'] = np.where(pa2017moddf['INT_COND'] == 'P' , 1 , pa2017moddf['INT_COND_RANK'] )      
pa2017moddf['INT_COND_RANK'] = np.where(pa2017moddf['INT_COND'] == 'F' , 2 , pa2017moddf['INT_COND_RANK'] )   
pa2017moddf['INT_COND_RANK'] = np.where(pa2017moddf['INT_COND'] == 'A' , 3 , pa2017moddf['INT_COND_RANK'] )
pa2017moddf['INT_COND_RANK'] = np.where(pa2017moddf['INT_COND'] == 'G' , 4 , pa2017moddf['INT_COND_RANK'] )
pa2017moddf['INT_COND_RANK'] = np.where(pa2017moddf['INT_COND'] == 'E' , 5 , pa2017moddf['INT_COND_RANK'] ) 

pa2017moddf['EXT_COND_RANK'] = NaN
pa2017moddf['EXT_COND_RANK'] = np.where(pa2017moddf['EXT_COND'] == 'P' , 1 , pa2017moddf['EXT_COND_RANK'] )      
pa2017moddf['EXT_COND_RANK'] = np.where(pa2017moddf['EXT_COND'] == 'F' , 2 , pa2017moddf['EXT_COND_RANK'] )   
pa2017moddf['EXT_COND_RANK'] = np.where(pa2017moddf['EXT_COND'] == 'A' , 3 , pa2017moddf['EXT_COND_RANK'] )
pa2017moddf['EXT_COND_RANK'] = np.where(pa2017moddf['EXT_COND'] == 'G' , 4 , pa2017moddf['EXT_COND_RANK'] )
pa2017moddf['EXT_COND_RANK'] = np.where(pa2017moddf['EXT_COND'] == 'E' , 5 , pa2017moddf['EXT_COND_RANK'] ) 

pa2017moddf['OVERALL_COND_RANK'] = NaN
pa2017moddf['OVERALL_COND_RANK'] = np.where(pa2017moddf['OVERALL_COND'] == 'P' , 1 , pa2017moddf['OVERALL_COND_RANK'] )      
pa2017moddf['OVERALL_COND_RANK'] = np.where(pa2017moddf['OVERALL_COND'] == 'F' , 2 , pa2017moddf['OVERALL_COND_RANK'] )   
pa2017moddf['OVERALL_COND_RANK'] = np.where(pa2017moddf['OVERALL_COND'] == 'A' , 3 , pa2017moddf['OVERALL_COND_RANK'] )
pa2017moddf['OVERALL_COND_RANK'] = np.where(pa2017moddf['OVERALL_COND'] == 'G' , 4 , pa2017moddf['OVERALL_COND_RANK'] )
pa2017moddf['OVERALL_COND_RANK'] = np.where(pa2017moddf['OVERALL_COND'] == 'E' , 5 , pa2017moddf['OVERALL_COND_RANK'] ) 

pa2017moddf['BTHRM_STYLE1_RANK'] = NaN
pa2017moddf['BTHRM_STYLE1_RANK'] = np.where(pa2017moddf['BTHRM_STYLE1'] == 'N' , 1 , pa2017moddf['BTHRM_STYLE1_RANK'] )      
pa2017moddf['BTHRM_STYLE1_RANK'] = np.where(pa2017moddf['BTHRM_STYLE1'] == 'S' , 2 , pa2017moddf['BTHRM_STYLE1_RANK'] )   
pa2017moddf['BTHRM_STYLE1_RANK'] = np.where(pa2017moddf['BTHRM_STYLE1'] == 'M' , 3 , pa2017moddf['BTHRM_STYLE1_RANK'] )
pa2017moddf['BTHRM_STYLE1_RANK'] = np.where(pa2017moddf['BTHRM_STYLE1'] == 'L' , 4 , pa2017moddf['BTHRM_STYLE1_RANK'] ) 

pa2017moddf['BTHRM_STYLE2_RANK'] = NaN
pa2017moddf['BTHRM_STYLE2_RANK'] = np.where(pa2017moddf['BTHRM_STYLE2'] == 'N' , 1 , pa2017moddf['BTHRM_STYLE2_RANK'] )      
pa2017moddf['BTHRM_STYLE2_RANK'] = np.where(pa2017moddf['BTHRM_STYLE2'] == 'S' , 2 , pa2017moddf['BTHRM_STYLE2_RANK'] )   
pa2017moddf['BTHRM_STYLE2_RANK'] = np.where(pa2017moddf['BTHRM_STYLE2'] == 'M' , 3 , pa2017moddf['BTHRM_STYLE2_RANK'] )
pa2017moddf['BTHRM_STYLE2_RANK'] = np.where(pa2017moddf['BTHRM_STYLE2'] == 'L' , 4 , pa2017moddf['BTHRM_STYLE2_RANK'] ) 

pa2017moddf['KITCHEN_STYLE1_RANK'] = NaN
pa2017moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2017moddf['KITCHEN_STYLE1'] == 'N' , 1 , pa2017moddf['KITCHEN_STYLE1_RANK'] )      
pa2017moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2017moddf['KITCHEN_STYLE1'] == 'S' , 2 , pa2017moddf['KITCHEN_STYLE1_RANK'] )   
pa2017moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2017moddf['KITCHEN_STYLE1'] == 'M' , 3 , pa2017moddf['KITCHEN_STYLE1_RANK'] )
pa2017moddf['KITCHEN_STYLE1_RANK'] = np.where(pa2017moddf['KITCHEN_STYLE1'] == 'L' , 4 , pa2017moddf['KITCHEN_STYLE1_RANK'] ) 

def rankchk(orig, rnk):
  print('\n============================================')
  print(orig)
  print('============================================\n')
  print(pa2017moddf[[rnk, orig]].value_counts().sort_index() )

rankchk('INT_COND'       ,'INT_COND_RANK')
rankchk('EXT_COND'       , 'EXT_COND_RANK')
rankchk('OVERALL_COND'   , 'OVERALL_COND_RANK')
rankchk('BTHRM_STYLE1'   , 'BTHRM_STYLE1_RANK')
rankchk('BTHRM_STYLE2'   , 'BTHRM_STYLE2_RANK')
rankchk('KITCHEN_STYLE1' , 'KITCHEN_STYLE1_RANK')


INT_COND

INT_COND_RANK  INT_COND
1.00           P              94
2.00           F            1206
3.00           A           44927
4.00           G           15927
5.00           E             504
dtype: int64

EXT_COND

EXT_COND_RANK  EXT_COND
1.00           P              85
2.00           F            2583
3.00           A           48858
4.00           G           11073
5.00           E              59
dtype: int64

OVERALL_COND

OVERALL_COND_RANK  OVERALL_COND
1.00               P                  72
2.00               F                 814
3.00               A               51666
4.00               G                9775
5.00               E                 331
dtype: int64

BTHRM_STYLE1

BTHRM_STYLE1_RANK  BTHRM_STYLE1
1.00               N                5194
2.00               S               38379
3.00               M               18463
4.00               L                 622
dtype: int64

BTHRM_STYLE2

BTHRM_STYLE2_RANK  BTHRM_STYLE2
1.00               N                43

### Drop the Original Columns (Keep the Ranks) And Rename All Columns

In [15]:
keepcols = [
  'PID'
, 'YR_REMODEL'
, 'RES_FLOOR'
, 'GROSS_AREA'
, 'LIVING_AREA'
, 'BED_RMS'
, 'FULL_BTH'
, 'HLF_BTH'
, 'KITCHEN'
, 'TT_RMS'
, 'INT_COND_RANK'
, 'EXT_COND_RANK'
, 'OVERALL_COND_RANK'
, 'BTHRM_STYLE1_RANK'
, 'BTHRM_STYLE2_RANK'
, 'KITCHEN_STYLE1_RANK' 
]
 

# Add '17' Suffix to All Column Names
keepcols17 = [sub + '_17' for sub in keepcols]

print(keepcols)
print(keepcols17)

# Create Final 2017 DF with Select Columns
pa2017finldf = pa2017moddf[keepcols]
pa2017finldf.columns = keepcols17
pa2017finldf.info()


['PID', 'YR_REMODEL', 'RES_FLOOR', 'GROSS_AREA', 'LIVING_AREA', 'BED_RMS', 'FULL_BTH', 'HLF_BTH', 'KITCHEN', 'TT_RMS', 'INT_COND_RANK', 'EXT_COND_RANK', 'OVERALL_COND_RANK', 'BTHRM_STYLE1_RANK', 'BTHRM_STYLE2_RANK', 'KITCHEN_STYLE1_RANK']
['PID_17', 'YR_REMODEL_17', 'RES_FLOOR_17', 'GROSS_AREA_17', 'LIVING_AREA_17', 'BED_RMS_17', 'FULL_BTH_17', 'HLF_BTH_17', 'KITCHEN_17', 'TT_RMS_17', 'INT_COND_RANK_17', 'EXT_COND_RANK_17', 'OVERALL_COND_RANK_17', 'BTHRM_STYLE1_RANK_17', 'BTHRM_STYLE2_RANK_17', 'KITCHEN_STYLE1_RANK_17']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 168181 entries, 0 to 170907
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   PID_17                  168181 non-null  object 
 1   YR_REMODEL_17           130070 non-null  float64
 2   RES_FLOOR_17            146436 non-null  float64
 3   GROSS_AREA_17           163926 non-null  float64
 4   LIVING_AREA_17          163926 non

# Merge/Add 2017 Features to 2022 File

In [16]:
print('pa2022 shape:', pa2022finldf.shape )
print('pa2017 shape:', pa2017finldf.shape )


pa2017finldf['PID_17b'] = pa2017finldf['PID_17'].str.replace('_','')
pa2017finldf['PID_17b'] = pa2017finldf['PID_17b'].astype('int64')
 
pa2022finldf['PID_22b'] = pa2022finldf['PID_22'].str.replace('_','')
pa2022finldf['PID_22b'] = pa2022finldf['PID_22b'].astype('int64')

pafinaldf = pa2022finldf.copy()
pafinaldf = pd.merge(pafinaldf, pa2017finldf, left_on='PID_22b', right_on="PID_17b", how='left') 
pafinaldf = pafinaldf.drop(['PID_17','PID_17b','PID_22b'], axis=1)
print('pafinal shape:', pafinaldf.shape )
print(pafinaldf.info() )

pa2022 shape: (25640, 17)
pa2017 shape: (168181, 16)
pafinal shape: (25640, 32)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 25640 entries, 0 to 25639
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PID_22                  25640 non-null  object 
 1   YR_REMODEL_22           10467 non-null  float64
 2   RES_FLOOR_22            25640 non-null  float64
 3   GROSS_AREA_22           25640 non-null  float64
 4   LIVING_AREA_22          25640 non-null  float64
 5   BED_RMS_22              25640 non-null  int64  
 6   FULL_BTH_22             25640 non-null  int64  
 7   HLF_BTH_22              25640 non-null  int64  
 8   KITCHEN_22              25640 non-null  int64  
 9   TT_RMS_22               25640 non-null  int64  
 10  INT_COND_RANK_22        25640 non-null  float64
 11  EXT_COND_RANK_22        25640 non-null  float64
 12  OVERALL_COND_RANK_22    25640 non-null  float64
 13  BTHRM_STYLE

## Calculate Change (2022-2017) and Set Flags

In [17]:
pafinaldf['RES_FLOOR_CHG_IND'] =  np.where(pafinaldf['RES_FLOOR_22'] > pafinaldf['RES_FLOOR_17']  , 1 , 0 )
pafinaldf['RES_FLOOR_CHG_IND'] =  np.where(pafinaldf['RES_FLOOR_22'] == NaN , NaN , pafinaldf['RES_FLOOR_CHG_IND']  )
pafinaldf['RES_FLOOR_CHG_IND'] =  np.where(pafinaldf['RES_FLOOR_17'] == NaN , NaN , pafinaldf['RES_FLOOR_CHG_IND']  )
pafinaldf['RES_FLOOR_CHG']     =  pafinaldf['RES_FLOOR_22'] - pafinaldf['RES_FLOOR_17']
pafinaldf['RES_FLOOR_CHG']     =  np.where(pafinaldf['RES_FLOOR_22'] == NaN , NaN , pafinaldf['RES_FLOOR_CHG']  )
pafinaldf['RES_FLOOR_CHG']     =  np.where(pafinaldf['RES_FLOOR_17'] == NaN , NaN , pafinaldf['RES_FLOOR_CHG']  )

pafinaldf['GROSS_AREA_CHG_IND'] =  np.where(pafinaldf['GROSS_AREA_22'] > pafinaldf['GROSS_AREA_17']  , 1 , 0 )
pafinaldf['GROSS_AREA_CHG_IND'] =  np.where(pafinaldf['GROSS_AREA_22'] == NaN , NaN , pafinaldf['GROSS_AREA_CHG_IND']  )
pafinaldf['GROSS_AREA_CHG_IND'] =  np.where(pafinaldf['GROSS_AREA_17'] == NaN , NaN , pafinaldf['GROSS_AREA_CHG_IND']  )
pafinaldf['GROSS_AREA_CHG']     =  pafinaldf['GROSS_AREA_22'] - pafinaldf['GROSS_AREA_17']
pafinaldf['GROSS_AREA_CHG']     =  np.where(pafinaldf['GROSS_AREA_22'] == NaN , NaN , pafinaldf['GROSS_AREA_CHG']  )
pafinaldf['GROSS_AREA_CHG']     =  np.where(pafinaldf['GROSS_AREA_17'] == NaN , NaN , pafinaldf['GROSS_AREA_CHG']  )

pafinaldf['LIVING_AREA_CHG_IND'] =  np.where(pafinaldf['LIVING_AREA_22'] > pafinaldf['LIVING_AREA_17']  , 1 , 0 )
pafinaldf['LIVING_AREA_CHG_IND'] =  np.where(pafinaldf['LIVING_AREA_22'] == NaN , NaN , pafinaldf['LIVING_AREA_CHG_IND']  )
pafinaldf['LIVING_AREA_CHG_IND'] =  np.where(pafinaldf['LIVING_AREA_17'] == NaN , NaN , pafinaldf['LIVING_AREA_CHG_IND']  )
pafinaldf['LIVING_AREA_CHG']     =  pafinaldf['LIVING_AREA_22'] - pafinaldf['LIVING_AREA_17']
pafinaldf['LIVING_AREA_CHG']     =  np.where(pafinaldf['LIVING_AREA_22'] == NaN , NaN , pafinaldf['LIVING_AREA_CHG']  )
pafinaldf['LIVING_AREA_CHG']     =  np.where(pafinaldf['LIVING_AREA_17'] == NaN , NaN , pafinaldf['LIVING_AREA_CHG']  )

pafinaldf['BED_RMS_CHG_IND'] =  np.where(pafinaldf['BED_RMS_22'] > pafinaldf['BED_RMS_17']  , 1 , 0 )
pafinaldf['BED_RMS_CHG_IND'] =  np.where(pafinaldf['BED_RMS_22'] == NaN , NaN , pafinaldf['BED_RMS_CHG_IND']  )
pafinaldf['BED_RMS_CHG_IND'] =  np.where(pafinaldf['BED_RMS_17'] == NaN , NaN , pafinaldf['BED_RMS_CHG_IND']  )
pafinaldf['BED_RMS_CHG']     =  pafinaldf['BED_RMS_22'] - pafinaldf['BED_RMS_17']
pafinaldf['BED_RMS_CHG']     =  np.where(pafinaldf['BED_RMS_22'] == NaN , NaN , pafinaldf['BED_RMS_CHG']  )
pafinaldf['BED_RMS_CHG']     =  np.where(pafinaldf['BED_RMS_17'] == NaN , NaN , pafinaldf['BED_RMS_CHG']  )

pafinaldf['TT_RMS_CHG_IND'] =  np.where(pafinaldf['TT_RMS_22'] > pafinaldf['TT_RMS_17']  , 1 , 0 )
pafinaldf['TT_RMS_CHG_IND'] =  np.where(pafinaldf['TT_RMS_22'] == NaN , NaN , pafinaldf['TT_RMS_CHG_IND']  )
pafinaldf['TT_RMS_CHG_IND'] =  np.where(pafinaldf['TT_RMS_17'] == NaN , NaN , pafinaldf['TT_RMS_CHG_IND']  )
pafinaldf['TT_RMS_CHG']     =  pafinaldf['TT_RMS_22'] - pafinaldf['TT_RMS_17']
pafinaldf['TT_RMS_CHG']     =  np.where(pafinaldf['TT_RMS_22'] == NaN , NaN , pafinaldf['TT_RMS_CHG']  )
pafinaldf['TT_RMS_CHG']     =  np.where(pafinaldf['TT_RMS_17'] == NaN , NaN , pafinaldf['TT_RMS_CHG']  )
  
pafinaldf['FULL_BTH_CHG_IND'] =  np.where(pafinaldf['FULL_BTH_22'] > pafinaldf['FULL_BTH_17']  , 1 , 0 )
pafinaldf['FULL_BTH_CHG_IND'] =  np.where(pafinaldf['FULL_BTH_22'] == NaN , NaN , pafinaldf['FULL_BTH_CHG_IND']  )
pafinaldf['FULL_BTH_CHG_IND'] =  np.where(pafinaldf['FULL_BTH_17'] == NaN , NaN , pafinaldf['FULL_BTH_CHG_IND']  )
pafinaldf['FULL_BTH_CHG']     =  pafinaldf['FULL_BTH_22'] - pafinaldf['FULL_BTH_17']
pafinaldf['FULL_BTH_CHG']     =  np.where(pafinaldf['FULL_BTH_22'] == NaN , NaN , pafinaldf['FULL_BTH_CHG']  )
pafinaldf['FULL_BTH_CHG']     =  np.where(pafinaldf['FULL_BTH_17'] == NaN , NaN , pafinaldf['FULL_BTH_CHG']  )

pafinaldf['HLF_BTH_CHG_IND'] =  np.where(pafinaldf['HLF_BTH_22'] > pafinaldf['HLF_BTH_17']  , 1 , 0 )
pafinaldf['HLF_BTH_CHG_IND'] =  np.where(pafinaldf['HLF_BTH_22'] == NaN , NaN , pafinaldf['HLF_BTH_CHG_IND']  )
pafinaldf['HLF_BTH_CHG_IND'] =  np.where(pafinaldf['HLF_BTH_17'] == NaN , NaN , pafinaldf['HLF_BTH_CHG_IND']  )
pafinaldf['HLF_BTH_CHG']     =  pafinaldf['HLF_BTH_22'] - pafinaldf['HLF_BTH_17']
pafinaldf['HLF_BTH_CHG']     =  np.where(pafinaldf['HLF_BTH_22'] == NaN , NaN , pafinaldf['HLF_BTH_CHG']  )
pafinaldf['HLF_BTH_CHG']     =  np.where(pafinaldf['HLF_BTH_17'] == NaN , NaN , pafinaldf['HLF_BTH_CHG']  )

pafinaldf['KITCHEN_CHG_IND'] =  np.where(pafinaldf['KITCHEN_22'] > pafinaldf['KITCHEN_17']  , 1 , 0 )
pafinaldf['KITCHEN_CHG_IND'] =  np.where(pafinaldf['KITCHEN_22'] == NaN , NaN , pafinaldf['KITCHEN_CHG_IND']  )
pafinaldf['KITCHEN_CHG_IND'] =  np.where(pafinaldf['KITCHEN_17'] == NaN , NaN , pafinaldf['KITCHEN_CHG_IND']  )
pafinaldf['KITCHEN_CHG']     =  pafinaldf['KITCHEN_22'] - pafinaldf['KITCHEN_17']
pafinaldf['KITCHEN_CHG']     =  np.where(pafinaldf['KITCHEN_22'] == NaN , NaN , pafinaldf['KITCHEN_CHG']  )
pafinaldf['KITCHEN_CHG']     =  np.where(pafinaldf['KITCHEN_17'] == NaN , NaN , pafinaldf['KITCHEN_CHG']  )
 
pafinaldf['INT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['INT_COND_RANK_22'] > pafinaldf['INT_COND_RANK_17']  , 1 , 0 )
pafinaldf['INT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['INT_COND_RANK_22'] == NaN , NaN , pafinaldf['INT_COND_RANK_CHG_IND']  )
pafinaldf['INT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['INT_COND_RANK_17'] == NaN , NaN , pafinaldf['INT_COND_RANK_CHG_IND']  )
pafinaldf['INT_COND_RANK_CHG']     =  pafinaldf['INT_COND_RANK_22'] - pafinaldf['INT_COND_RANK_17']
pafinaldf['INT_COND_RANK_CHG']     =  np.where(pafinaldf['INT_COND_RANK_22'] == NaN , NaN , pafinaldf['INT_COND_RANK_CHG']  )
pafinaldf['INT_COND_RANK_CHG']     =  np.where(pafinaldf['INT_COND_RANK_17'] == NaN , NaN , pafinaldf['INT_COND_RANK_CHG']  )

pafinaldf['EXT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['EXT_COND_RANK_22'] > pafinaldf['EXT_COND_RANK_17']  , 1 , 0 )
pafinaldf['EXT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['EXT_COND_RANK_22'] == NaN , NaN , pafinaldf['EXT_COND_RANK_CHG_IND']  )
pafinaldf['EXT_COND_RANK_CHG_IND'] =  np.where(pafinaldf['EXT_COND_RANK_17'] == NaN , NaN , pafinaldf['EXT_COND_RANK_CHG_IND']  )
pafinaldf['EXT_COND_RANK_CHG']     =  pafinaldf['EXT_COND_RANK_22'] - pafinaldf['EXT_COND_RANK_17']
pafinaldf['EXT_COND_RANK_CHG']     =  np.where(pafinaldf['EXT_COND_RANK_22'] == NaN , NaN , pafinaldf['EXT_COND_RANK_CHG']  )
pafinaldf['EXT_COND_RANK_CHG']     =  np.where(pafinaldf['EXT_COND_RANK_17'] == NaN , NaN , pafinaldf['EXT_COND_RANK_CHG']  )

pafinaldf['OVERALL_COND_RANK_CHG_IND'] =  np.where(pafinaldf['OVERALL_COND_RANK_22'] > pafinaldf['OVERALL_COND_RANK_17']  , 1 , 0 )
pafinaldf['OVERALL_COND_RANK_CHG_IND'] =  np.where(pafinaldf['OVERALL_COND_RANK_22'] == NaN , NaN , pafinaldf['OVERALL_COND_RANK_CHG_IND']  )
pafinaldf['OVERALL_COND_RANK_CHG_IND'] =  np.where(pafinaldf['OVERALL_COND_RANK_17'] == NaN , NaN , pafinaldf['OVERALL_COND_RANK_CHG_IND']  )
pafinaldf['OVERALL_COND_RANK_CHG']     =  pafinaldf['OVERALL_COND_RANK_22'] - pafinaldf['OVERALL_COND_RANK_17']
pafinaldf['OVERALL_COND_RANK_CHG']     =  np.where(pafinaldf['OVERALL_COND_RANK_22'] == NaN , NaN , pafinaldf['OVERALL_COND_RANK_CHG']  )
pafinaldf['OVERALL_COND_RANK_CHG']     =  np.where(pafinaldf['OVERALL_COND_RANK_17'] == NaN , NaN , pafinaldf['OVERALL_COND_RANK_CHG']  )

pafinaldf['BTHRM_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE1_RANK_22'] > pafinaldf['BTHRM_STYLE1_RANK_17']  , 1 , 0 )
pafinaldf['BTHRM_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE1_RANK_22'] == NaN , NaN , pafinaldf['BTHRM_STYLE1_RANK_CHG_IND']  )
pafinaldf['BTHRM_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE1_RANK_17'] == NaN , NaN , pafinaldf['BTHRM_STYLE1_RANK_CHG_IND']  )
pafinaldf['BTHRM_STYLE1_RANK_CHG']     =  pafinaldf['BTHRM_STYLE1_RANK_22'] - pafinaldf['BTHRM_STYLE1_RANK_17']
pafinaldf['BTHRM_STYLE1_RANK_CHG']     =  np.where(pafinaldf['BTHRM_STYLE1_RANK_22'] == NaN , NaN , pafinaldf['BTHRM_STYLE1_RANK_CHG']  )
pafinaldf['BTHRM_STYLE1_RANK_CHG']     =  np.where(pafinaldf['BTHRM_STYLE1_RANK_17'] == NaN , NaN , pafinaldf['BTHRM_STYLE1_RANK_CHG']  )

pafinaldf['BTHRM_STYLE2_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE2_RANK_22'] > pafinaldf['BTHRM_STYLE2_RANK_17']  , 1 , 0 )
pafinaldf['BTHRM_STYLE2_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE2_RANK_22'] == NaN , NaN , pafinaldf['BTHRM_STYLE2_RANK_CHG_IND']  )
pafinaldf['BTHRM_STYLE2_RANK_CHG_IND'] =  np.where(pafinaldf['BTHRM_STYLE2_RANK_17'] == NaN , NaN , pafinaldf['BTHRM_STYLE2_RANK_CHG_IND']  )
pafinaldf['BTHRM_STYLE2_RANK_CHG']     =  pafinaldf['BTHRM_STYLE2_RANK_22'] - pafinaldf['BTHRM_STYLE2_RANK_17']
pafinaldf['BTHRM_STYLE2_RANK_CHG']     =  np.where(pafinaldf['BTHRM_STYLE2_RANK_22'] == NaN , NaN , pafinaldf['BTHRM_STYLE2_RANK_CHG']  )
pafinaldf['BTHRM_STYLE2_RANK_CHG']     =  np.where(pafinaldf['BTHRM_STYLE2_RANK_17'] == NaN , NaN , pafinaldf['BTHRM_STYLE2_RANK_CHG']  )

pafinaldf['KITCHEN_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['KITCHEN_STYLE1_RANK_22'] > pafinaldf['KITCHEN_STYLE1_RANK_17']  , 1 , 0 )
pafinaldf['KITCHEN_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['KITCHEN_STYLE1_RANK_22'] == NaN , NaN , pafinaldf['KITCHEN_STYLE1_RANK_CHG_IND']  )
pafinaldf['KITCHEN_STYLE1_RANK_CHG_IND'] =  np.where(pafinaldf['KITCHEN_STYLE1_RANK_17'] == NaN , NaN , pafinaldf['KITCHEN_STYLE1_RANK_CHG_IND']  )
pafinaldf['KITCHEN_STYLE1_RANK_CHG']     =  pafinaldf['KITCHEN_STYLE1_RANK_22'] - pafinaldf['KITCHEN_STYLE1_RANK_17']
pafinaldf['KITCHEN_STYLE1_RANK_CHG']     =  np.where(pafinaldf['KITCHEN_STYLE1_RANK_22'] == NaN , NaN , pafinaldf['KITCHEN_STYLE1_RANK_CHG']  )
pafinaldf['KITCHEN_STYLE1_RANK_CHG']     =  np.where(pafinaldf['KITCHEN_STYLE1_RANK_17'] == NaN , NaN , pafinaldf['KITCHEN_STYLE1_RANK_CHG']  )


In [18]:
# Check Results on Count Features

def flagchk(col22, col12, colind):
  print('\n============================================')
  print(colind)
  print('============================================\n')
  print(pafinaldf[[colind]].value_counts().sort_index(), '\n' )
  print(pafinaldf[[colind, col12, col22]].value_counts().sort_index() )
 
   
flagchk('RES_FLOOR_22', 'RES_FLOOR_17', 'RES_FLOOR_CHG_IND')
flagchk('TT_RMS_22', 'TT_RMS_17', 'TT_RMS_CHG_IND')
flagchk('FULL_BTH_22', 'FULL_BTH_17', 'FULL_BTH_CHG_IND')
flagchk('HLF_BTH_22', 'HLF_BTH_17', 'HLF_BTH_CHG_IND')
flagchk('BED_RMS_22', 'BED_RMS_17', 'BED_RMS_CHG_IND')
flagchk('KITCHEN_22', 'KITCHEN_17', 'KITCHEN_CHG_IND')
flagchk('INT_COND_RANK_22', 'INT_COND_RANK_17', 'INT_COND_RANK_CHG_IND')
flagchk('EXT_COND_RANK_22', 'EXT_COND_RANK_17', 'EXT_COND_RANK_CHG_IND')
flagchk('OVERALL_COND_RANK_22', 'OVERALL_COND_RANK_17', 'OVERALL_COND_RANK_CHG_IND')
flagchk('BTHRM_STYLE1_RANK_22', 'BTHRM_STYLE1_RANK_17', 'BTHRM_STYLE1_RANK_CHG_IND')
flagchk('BTHRM_STYLE2_RANK_22', 'BTHRM_STYLE2_RANK_17', 'BTHRM_STYLE2_RANK_CHG_IND')
flagchk('KITCHEN_STYLE1_RANK_22', 'KITCHEN_STYLE1_RANK_17', 'KITCHEN_STYLE1_RANK_CHG_IND')
 
 


RES_FLOOR_CHG_IND

RES_FLOOR_CHG_IND
0.00                 25336
1.00                   304
dtype: int64 

RES_FLOOR_CHG_IND  RES_FLOOR_17  RES_FLOOR_22
0.00               1.00          1.00             4664
                   1.50          1.00                8
                                 1.50             2842
                   2.00          1.00                1
                                 1.50                6
                                 2.00            14730
                   2.50          2.00                7
                                 2.50             1388
                   3.00          2.50                4
                                 3.00             1198
                   3.50          3.00                2
                                 3.50               78
                   4.00          3.50                1
                                 4.00              220
                   4.50          4.50               16
                   5.0

In [19]:
# Check Results on Continuous Features

def flagnumchk(colchg, colind):
  print('\n============================================')
  print(colind)
  print('============================================\n')
  print(pafinaldf[[colind]].value_counts().sort_index(), '\n' )
  print('Indicator = 1\n')
  print(pafinaldf[pafinaldf[colind] == 1][colchg].describe() )
  print('\nIndicator = 0\n')
  print(pafinaldf[pafinaldf[colind] == 0][colchg].describe() )


flagnumchk('GROSS_AREA_CHG' , 'GROSS_AREA_CHG_IND')
flagnumchk('LIVING_AREA_CHG' , 'LIVING_AREA_CHG_IND')


GROSS_AREA_CHG_IND

GROSS_AREA_CHG_IND
0.00                  24120
1.00                   1520
dtype: int64 

Indicator = 1

count   1520.00
mean     544.43
std      926.62
min        0.75
25%       72.00
50%      215.80
75%      538.30
max     7229.20
Name: GROSS_AREA_CHG, dtype: float64

Indicator = 0

count   24063.00
mean     -175.89
std       222.27
min     -2484.00
25%      -331.20
50%       -35.60
75%         0.00
max         0.00
Name: GROSS_AREA_CHG, dtype: float64

LIVING_AREA_CHG_IND

LIVING_AREA_CHG_IND
0.00                   20493
1.00                    5147
dtype: int64 

Indicator = 1

count   5147.00
mean     135.80
std      396.71
min        0.05
25%        0.20
50%        0.40
75%       57.10
max     4938.20
Name: LIVING_AREA_CHG, dtype: float64

Indicator = 0

count   20436.00
mean       -4.46
std        45.33
min     -1889.00
25%         0.00
50%         0.00
75%         0.00
max         0.00
Name: LIVING_AREA_CHG, dtype: float64


## Add Remodel Flags and Check Results

In [20]:
pafinaldf['REMODEL_EVER_FLAG'] = np.where(pafinaldf['YR_REMODEL_22'] > 0 , 1, 0)
pafinaldf['REMODEL_LAST_5YR'] = np.where(pafinaldf['YR_REMODEL_22'] > 2017 , 1, 0)

In [21]:
# Check Improvement Flags by Remodel Ever Flag

def impchk(impcol):
  print('\n============================================')
  print(impcol)
  print('============================================\n')
  print(pafinaldf[['REMODEL_EVER_FLAG', impcol]].value_counts().sort_index() )

impchk('RES_FLOOR_CHG_IND')
impchk('TT_RMS_CHG_IND')
impchk('FULL_BTH_CHG_IND') 
impchk('HLF_BTH_CHG_IND') 
impchk('BED_RMS_CHG_IND')
impchk('KITCHEN_CHG_IND')
impchk('GROSS_AREA_CHG_IND')
impchk('LIVING_AREA_CHG_IND') 
impchk('INT_COND_RANK_CHG_IND') 
impchk('EXT_COND_RANK_CHG_IND')
impchk('OVERALL_COND_RANK_CHG_IND')
impchk('BTHRM_STYLE1_RANK_CHG_IND')
impchk('BTHRM_STYLE2_RANK_CHG_IND')
impchk('KITCHEN_STYLE1_RANK_CHG_IND')


RES_FLOOR_CHG_IND

REMODEL_EVER_FLAG  RES_FLOOR_CHG_IND
0                  0.00                 15118
                   1.00                    55
1                  0.00                 10218
                   1.00                   249
dtype: int64

TT_RMS_CHG_IND

REMODEL_EVER_FLAG  TT_RMS_CHG_IND
0                  0.00              14881
                   1.00                292
1                  0.00               9711
                   1.00                756
dtype: int64

FULL_BTH_CHG_IND

REMODEL_EVER_FLAG  FULL_BTH_CHG_IND
0                  0.00                14892
                   1.00                  281
1                  0.00                 9396
                   1.00                 1071
dtype: int64

HLF_BTH_CHG_IND

REMODEL_EVER_FLAG  HLF_BTH_CHG_IND
0                  0.00               15008
                   1.00                 165
1                  0.00                9952
                   1.00                 515
dtype: int64

BED_RMS_CHG_IND

RE

In [22]:
# Check Improvement Flags by Remodel Last 5 Year Flag

def impchk(impcol):
  print('\n============================================')
  print(impcol)
  print('============================================\n')
  print(pafinaldf[['REMODEL_LAST_5YR', impcol]].value_counts().sort_index() )

impchk('RES_FLOOR_CHG_IND')
impchk('TT_RMS_CHG_IND')
impchk('FULL_BTH_CHG_IND') 
impchk('HLF_BTH_CHG_IND') 
impchk('BED_RMS_CHG_IND')
impchk('KITCHEN_CHG_IND')
impchk('GROSS_AREA_CHG_IND')
impchk('LIVING_AREA_CHG_IND') 
impchk('INT_COND_RANK_CHG_IND') 
impchk('EXT_COND_RANK_CHG_IND')
impchk('OVERALL_COND_RANK_CHG_IND')
impchk('BTHRM_STYLE1_RANK_CHG_IND')
impchk('BTHRM_STYLE2_RANK_CHG_IND')
impchk('KITCHEN_STYLE1_RANK_CHG_IND')


RES_FLOOR_CHG_IND

REMODEL_LAST_5YR  RES_FLOOR_CHG_IND
0                 0.00                 24064
                  1.00                   189
1                 0.00                  1272
                  1.00                   115
dtype: int64

TT_RMS_CHG_IND

REMODEL_LAST_5YR  TT_RMS_CHG_IND
0                 0.00              23468
                  1.00                785
1                 0.00               1124
                  1.00                263
dtype: int64

FULL_BTH_CHG_IND

REMODEL_LAST_5YR  FULL_BTH_CHG_IND
0                 0.00                23390
                  1.00                  863
1                 0.00                  898
                  1.00                  489
dtype: int64

HLF_BTH_CHG_IND

REMODEL_LAST_5YR  HLF_BTH_CHG_IND
0                 0.00               23794
                  1.00                 459
1                 0.00                1166
                  1.00                 221
dtype: int64

BED_RMS_CHG_IND

REMODEL_LAST_5YR  BED_

# Export Final File

In [23]:
pafinaldf.to_csv('pa_home_improvements_2017_2022.csv', header=True, index=False)